In [3]:
import pandas as pd

df = pd.read_csv("Titanic-Dataset.csv")
df.head(10)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [43]:
women_survived = df[(df['Sex'] == 'female') & (df['Survived'] == 1)]['Age'].dropna()
women_not_survived = df[(df['Sex'] == 'female') & (df['Survived'] == 0)]['Age'].dropna()
men_survived = df[(df['Sex'] == 'male') & (df['Survived'] == 1)]['Age'].dropna()
men_not_survived = df[(df['Sex'] == 'male') & (df['Survived'] == 0)]['Age'].dropna()

In [44]:
means_age = []
n = 1

while n < 500:
    n += 1
    sample = df[(df['Sex'] == 'female') & (df['Survived'] == 1)].sample(30)['Age'].dropna().mean()
    means_age.append(sample)


In [45]:
from scipy.stats import shapiro
shapiro(means_age)

ShapiroResult(statistic=np.float64(0.9939131003419375), pvalue=np.float64(0.04287561900050781))

In [46]:
means_age = []
n = 1

while n < 500:
    n += 1
    sample = df[(df['Sex'] == 'male') & (df['Survived'] == 1)].sample(30)['Age'].dropna().mean()
    means_age.append(sample)

In [47]:
shapiro(means_age)

ShapiroResult(statistic=np.float64(0.9980407834053957), pvalue=np.float64(0.8429094291676208))

In [48]:
means_age = []
n = 1

while n < 500:
    n += 1
    sample = df[(df['Sex'] == 'female') & (df['Survived'] == 0)].sample(30)['Age'].dropna().mean()
    means_age.append(sample)

In [49]:
shapiro(means_age)

ShapiroResult(statistic=np.float64(0.9940867063315154), pvalue=np.float64(0.04960478140760157))

In [50]:
means_age = []
n = 1

while n < 500:
    n += 1
    sample = df[(df['Sex'] == 'male') & (df['Survived'] == 0)].sample(30)['Age'].dropna().mean()
    means_age.append(sample)

In [51]:
shapiro(means_age)

ShapiroResult(statistic=np.float64(0.994872321730974), pvalue=np.float64(0.0957659217255438))

In [53]:
from scipy.stats import levene

w_s = df[(df['Sex'] == 'female') & (df['Survived'] == 1)].sample(30)['Age'].dropna()
w_n_s = df[(df['Sex'] == 'female') & (df['Survived'] == 0)].sample(30)['Age'].dropna()
m_s = df[(df['Sex'] == 'male') & (df['Survived'] == 1).sample(30)]['Age'].dropna()
m_n_s = df[(df['Sex'] == 'male') & (df['Survived'] == 0).sample(30)]['Age'].dropna()
levene(women_survived,women_not_survived,men_survived,men_not_survived)

LeveneResult(statistic=np.float64(1.0646663998496362), pvalue=np.float64(0.36338158705109413))

In [55]:
from scipy.stats import f_oneway

f_oneway(women_survived, women_not_survived, men_survived, men_not_survived)

F_onewayResult(statistic=np.float64(5.47124408097316), pvalue=np.float64(0.0010132959881560411))

In [54]:
levene(w_s,w_n_s,m_s,m_n_s)

LeveneResult(statistic=np.float64(0.8574622039861639), pvalue=np.float64(0.4680935123198442))

In [56]:
f_oneway(w_s, w_n_s, m_s, m_n_s)

F_onewayResult(statistic=np.float64(0.7930078992430907), pvalue=np.float64(0.5024920732300246))